In [ ]:
#default_exp prodigy_demo

In [1]:
#no_test

#data structure imports
import pandas as pd
import numpy as np

#python imports
import random

#modeling imports
from spacy.util import fix_random_seed
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.preprocessing import *
from ssda_nlp.model_performance_utils import *

In [2]:
#no_test
seed = 2436
random.seed(seed)
fix_random_seed(seed)

In [12]:
import json

with open('C:\\Users\\genkindn\\means_nlp\\annotated_json\\750003.json', encoding="utf-8") as infile:    
    annotation = json.load(infile)

ordered_annotation = []

with open('C:\\Users\\genkindn\\means_nlp\\lighttag_jsonl\\750003\\750003.jsonl', encoding="utf-8") as infile:
    for line in infile:
        text = line[10:line.find('}') - 1]
        ordered_annotation.append({'text': text})
        ordered_annotation[len(ordered_annotation) - 1]['annotations'] = []
        for ann_line in annotation['examples']:
            if text == ann_line['content'] and 'annotations' in ann_line:
                for annot in ann_line['annotations']:
                    ordered_annotation[len(ordered_annotation) - 1]['annotations'].append(annot)

annotation_dict_list = []
index = 1

for line in ordered_annotation:    
    for annotation in line['annotations']:
        annotation_dict_list.append({'entry_no': index})
        annotation_dict_list[len(annotation_dict_list) - 1]['text'] = line['text']
        annotation_dict_list[len(annotation_dict_list) - 1]['entity'] = annotation['value']
        annotation_dict_list[len(annotation_dict_list) - 1]['start'] = annotation['start']
        annotation_dict_list[len(annotation_dict_list) - 1]['end'] = annotation['end']
        annotation_dict_list[len(annotation_dict_list) - 1]['label'] = annotation['tag']
    index += 1
    
annotations = []
duplicates = []

for count, annotation in enumerate(annotation_dict_list):
    if not annotation in annotations:        
        annotations.append(annotation)
    else:        
        duplicates.append(count)

fixed_list = []

for count, annotation in enumerate(annotation_dict_list):
    if not count in duplicates:        
        fixed_list.append(annotation)
        
collated_df = pd.DataFrame(fixed_list)
collated_df.head(50)

,entry_no,text,entity,start,end,label
0,1,Saturday 8 January 1870 Said to be the coldest...,January,11,18,Date
1,1,Saturday 8 January 1870 Said to be the coldest...,Saturday 8 January 1870,0,23,Date
2,2,coldest day throughout of the winter thus far....,Willie,47,53,Name
3,3,"Sibby home, returned 3 P.M. feeling very frosty.",Sibby,0,5,Name
4,4,Sunday 9 Clear & very cold. Mercury down nearl...,Sunday 9,0,8,Date
5,7,Phipps' pew. Jeff called & bid goodbye. Willie...,Phipps',0,7,Name
6,7,Phipps' pew. Jeff called & bid goodbye. Willie...,Jeff,13,17,Name
7,7,Phipps' pew. Jeff called & bid goodbye. Willie...,Willie,40,46,Name
8,8,Frank Powel.,Powel,6,11,Name
9,8,Frank Powel.,Frank,0,5,Name


# First Book of Baptisms, Matanzas (15834)

In [13]:
#no_test

#collated_df = prodigy_output_to_collated_df("transcriptions//15834_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)

for index, element in enumerate(train_spacy):    
    unique_entities = []
    covered_tokens = []
    for entity in element[1]["entities"]:
        covered_span = range(entity[0], entity[1] + 1)
        overlap = False
        for token in covered_span:            
            if token in covered_tokens:
                overlap = True
                break
        if (entity not in unique_entities) and (not overlap):
            unique_entities.append(entity)
            for token in covered_span:
                covered_tokens.append(token)
    entities = []
    for entity in unique_entities:
        entities.append(entity)
    dic = {"entities": entities}
    text = element[0]
    train_spacy[index] = (text, dic)
    
train_spacy

# Look at column names
#pd.set_option('display.max_rows', None)
#train_df

[('Saturday 8 January 1870 Said to be the coldest morning &',
  {'entities': [(11, 18, 'Date')]}),
 ('coldest day throughout of the winter thus far. Willie took',
  {'entities': [(47, 53, 'Name')]}),
 ('Sibby home, returned 3 P.M. feeling very frosty.',
  {'entities': [(0, 5, 'Name')]}),
 ('Monday 10 Hard freeze succeeded by warm sun. Snow melting',
  {'entities': [(0, 9, 'Date')]}),
 ('to Yellow Store. Registered letter for Watterson to Tutwiler,',
  {'entities': [(39, 48, 'Name'), (52, 61, 'Name'), (3, 15, 'Location')]}),
 ("to Leslie's Illustrated for Hoffman. Mailed Willie's letter",
  {'entities': [(28, 35, 'Name'), (44, 50, 'Name')]}),
 ("to Frank and one to Monk. Called at Captain Lyons' & carried",
  {'entities': [(3, 8, 'Name'),
    (20, 24, 'Name'),
    (44, 49, 'Name'),
    (36, 43, 'Name')]}),
 ('down mail for Colonel Cocke. Sue Miller came up in company',
  {'entities': [(14, 27, 'Name'), (29, 39, 'Name')]}),
 ("on way to Button's. Stopped a moment at J.M. Lyons' and Misse

In [10]:
#no_test

train_spacy[:2]

[('Surgoinsville June 23 1908',
  {'entities': [(0, 13, 'Location'), (14, 26, 'Date')]}),
 ('My Dear Mother,', {'entities': [(8, 14, 'Relationship')]})]

In [14]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/15834'

#load model from pretrained
nlp_model = load_model()

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

C:\Users\genkindn\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "& hauling lumber for Ben's cabin. Watterson & Bake..." with entities "[(34, 43, 'Name'), (20, 26, 'Name'), (46, 51, 'Nam...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,181.503511,0.0,0.0,0.0


Saved model to models\15834


C:\Users\genkindn\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "& hauling lumber for Ben's cabin. Watterson & Bake..." with entities "[(34, 43, 'Name'), (20, 26, 'Name'), (46, 51, 'Nam...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,82.32178,0.0,0.0,0.0


...

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,12.0,18.016402,38.461538,31.25,34.482759


Performance hasn't improved for 4 cycles...


C:\Users\genkindn\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "& hauling lumber for Ben's cabin. Watterson & Bake..." with entities "[(34, 43, 'Name'), (20, 26, 'Name'), (46, 51, 'Nam...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,13.0,17.959951,42.857143,37.5,40.0


Performance hasn't improved for 5 cycles...
Done training after 13 meta cycles.


# Second Book of Baptisms, St. Augustine (239746)

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/239746_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)

# Look at column names
train_df.head(10)

,entry_no,text,entity,start,end,label
31,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Antonio Guillo,3,17,PER
32,2,"2. Antonio Guillo Miercoles, día veinte de Nov...","Miercoles, día veinte de Noviembre de Mil Sete...",18,86,DATE
33,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Don Miguel o’Reilly,90,109,PER
34,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Teniente de Cura Beneficiado,110,138,CHAR
35,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Vicario,140,147,CHAR
36,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Juez Eclesiás tico Auxiliar,151,178,CHAR
37,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Iglesia parroquial,187,205,LOC
38,2,"2. Antonio Guillo Miercoles, día veinte de Nov...","provincia, de San Agustín",208,233,LOC
39,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Florida Oriental,240,256,LOC
40,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",niño,295,299,CHAR


In [ ]:
#no_test

train_spacy[:2]

[('2. Antonio Guillo Miercoles, día veinte de Noviembre de Mil Setecientos Noventa y Tres Yo Don Miguel o’Reilly Teniente de Cura Beneficiado, Vicario, y Juez Eclesiás tico Auxiliar de esta Iglesia parroquial y provincia, de San Agustín de la Florida Oriental. Baptisé y puse los santos oleos a un niño de color, moreno, que nacio el día primero de octubre del presente año, hijo na tural de los negros libres Isaac Guillo y Sara Ca mel, ambos naturales de America del Norte; y y en el exerci las sacras Ceremonias, y preces de nuestra Santa madre Iglesia, y la puse por nombre Juan Antonio. Fueron sus padrinos Antonio Pellice, y Susana Pellicer vecinos de esta Ciudad, les advertí el parentesco espiritual, y demás obligaciones, y lo firmé en dicho dia, mes y año.  Miguel OReilly ',
  {'entities': [(3, 17, 'PER'),
    (18, 86, 'DATE'),
    (90, 109, 'PER'),
    (110, 138, 'CHAR'),
    (140, 147, 'CHAR'),
    (151, 178, 'CHAR'),
    (187, 205, 'LOC'),
    (208, 233, 'LOC'),
    (240, 256, 'LOC'

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/239746'

#load model from pretrained
nlp_model = load_model('es_core_news_md')

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,3058.593277,0.673077,0.322284,0.435866


Saved model to models\239746


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,1895.333521,0.768,0.530387,0.627451


Saved model to models\239746


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,3.0,1568.612363,0.838403,0.812155,0.82507


...

Performance hasn't improved for 3 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,28.0,287.60354,0.890741,0.88582,0.888273


Performance hasn't improved for 4 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,29.0,277.603007,0.876838,0.878453,0.877645


Performance hasn't improved for 5 cycles...
Done training after 29 meta cycles.


It's interesting how similar this training performance was to the Matanzas volume above. If this is replicated across more data sets, it probably means something (although I'm not sure what exactly).

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/166470_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.9, prop_validation = 0.05, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(collated_df)

# Look at column names
train_spacy[:5]

[('No. 1 Roque Cruz En catorce de Agosto de mil ochocientos treintaynuebe: yo Don Antonio Loreto Sanchez Presbitero cura beneficiado por Su Majestad de la Iglesia Parroquial de la Purisima concepcion de esta villa de Cienfuegos en ella y su jurisdiccion Vicario Eclesiastico por Santa Eclesiastica Illustrisima bautice solemnemente y puse los santos oleos a un negro adulto natural de Guinea de la propiedad de Don Agustin de Santa Cruz: en cuyo negro ejerci las sacras ceremonias y preces y le puse por nombre Roque fueron sus padrinos Jose y Monserrate; aquien adverti el parentesco espiritual y demas obligaciones y lo firme Antonio L. Sanchez ',
  {'entities': [(6, 16, 'PER'),
    (20, 70, 'DATE'),
    (75, 101, 'PER'),
    (113, 117, 'CHAR'),
    (152, 196, 'LOC'),
    (205, 224, 'LOC'),
    (251, 271, 'CHAR'),
    (358, 363, 'CHAR'),
    (364, 370, 'CHAR'),
    (371, 378, 'CHAR'),
    (382, 388, 'LOC'),
    (395, 404, 'CHAR'),
    (408, 433, 'PER'),
    (443, 448, 'CHAR'),
    (508, 513,

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/166470'

#load model from pretrained
nlp_model = load_model()

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,2859.259378,0.0,0.0,0.0


Saved model to models\166470


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,1781.561883,43.902439,41.860465,42.857143


Saved model to models\166470


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,3.0,1313.44701,60.97561,58.139535,59.52381


...

Performance hasn't improved for 3 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,12.0,125.943441,100.0,100.0,100.0


Performance hasn't improved for 4 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,13.0,97.236912,100.0,100.0,100.0


Performance hasn't improved for 5 cycles...
Done training after 13 meta cycles.
